In [1]:
!pip install datasets .

Processing /Users/frascuchon/Projects/argilla/argilla/argilla
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached argilla_sdk-2.0.0a0.dev0-py3-none-any.whl
  Using cached argilla_v1-1.29.0a1-py3-none-any.whl
  Created wheel for argilla: filename=argilla-2.0.0-py3-none-any.whl size=1343 sha256=43a8fb52aad7a5a29e15c5a152bd463ba4619314a66bf0bb5d61eb39772b126f
  Stored in directory: /private/var/folders/nv/jy1fzkzj4zb7bq12rrsksly00000gn/T/pip-ephem-wheel-cache-zuuhz1g0/wheels/a9/d4/42/8bd711fdc668e8088250b6e0a2083d611916233f5c3238eeb3
Successfully built argilla
  Attempting uninstall: argilla-v1
    Found existing installation: argilla-v1 1.29.0a1
    Uninstalling argilla-v1-1.29.0a1:
      Successfully uninstalled argilla-v1-1.29.0a1
  Attempting uninstall: argilla-sdk
    Found existing installation: argilla-sdk 2.0.0a0.dev0
    Uninstalling argilla-sdk-2.0.0a0.dev0:
      Successfully unins

In [2]:
import argilla.v1 as rg_v1

In [4]:
import os
url = "https://demo.argilla.io"
api_key = "<your-api-key>"

In [5]:
rg_v1.init(url, api_key)

/Users/frascuchon/.pyenv/versions/3.9.18/envs/argilla-v2/lib/python3.9/site-packages/argilla_v1/client/client.py:178: UserWarning: No workspace configuration was detected. To work with Argilla datasets, specify a valid workspace name on `rg.init` or set it up through the `rg.set_workspace` function.
  warnings.warn(


In [6]:
dataset_name = "news-programmatic-labeling"
workspace = "demo"

Read the current dataset labels

In [7]:
settings_v1 = rg_v1.load_dataset_settings(dataset_name, workspace)
settings_v1

TextClassificationSettings(label_schema=['Sci/Tech', 'Sports', 'World', 'Business'])

Read the dataset records


In [8]:
hf_records = rg_v1.load(dataset_name, workspace, limit=100, query="annotated_by:f2c172449344").to_datasets()
hf_records

Dataset({
    features: ['text', 'inputs', 'prediction', 'prediction_agent', 'annotation', 'annotation_agent', 'vectors', 'multi_label', 'explanation', 'id', 'metadata', 'status', 'event_timestamp', 'metrics'],
    num_rows: 100
})

In [9]:
hf_records[0]

{'text': 'Russians admit airliner bombing blunder Russian security forces were facing further criticism last night after it was revealed that the two female Chechen suicide bombers who destroyed two planes in August with the loss ',
 'inputs': {'text': 'Russians admit airliner bombing blunder Russian security forces were facing further criticism last night after it was revealed that the two female Chechen suicide bombers who destroyed two planes in August with the loss '},
 'prediction': None,
 'prediction_agent': None,
 'annotation': 'World',
 'annotation_agent': 'f2c172449344',
 'vectors': {'mini-lm-sentence-transformers': [0.04042864963412285,
   0.04933001473546028,
   -0.044904328882694244,
   0.11481531709432602,
   0.08239579200744629,
   0.03636464104056358,
   0.032279059290885925,
   0.005122746340930462,
   0.006479339208453894,
   0.042862217873334885,
   -0.04830484464764595,
   0.02159479819238186,
   -0.021633388474583626,
   0.06240358576178551,
   -0.043455082923173904

In [10]:
import argilla as rg

client = rg.Argilla() # create the new dataset into a different argilla server instance 

In [11]:
settings = rg.Settings(
    fields=[
        # The default name for text classification is `text`,
        #  but we should provide all names included in `record.inputs`
        rg.TextField(name="text"),
    ],
    questions=[
        # The basis question for text classification is a LabelQuestion for single-label
        # or MultiLabelQuestion for multi-label classification
        rg.LabelQuestion(name="label", labels=settings_v1.label_schema),
    ],
    metadata=[
        # Here, we need to provide all relevant metadata fields.
        rg.TermsMetadataProperty(name="split"),
    ],
    vectors=[
        # The vectors fields available in the dataset
        rg.VectorField(name='mini-lm-sentence-transformers', dimensions=384),
    ],
)
settings

Settings(guidelines=None, allow_extra_metadata=False, fields=<argilla_sdk.settings._resource.SettingsProperties object at 0x1252e4640>, questions=[LabelQuestion(name=label, title=label, description=label, type=label_selection, required=True) 
], vectors=<argilla_sdk.settings._resource.SettingsProperties object at 0x1252dcfa0>, metadata=<argilla_sdk.settings._resource.SettingsProperties object at 0x1252e42e0>)

In [12]:
ds = client.datasets(name=dataset_name)
if ds.exists():
    ds.delete()

dataset = rg.Dataset(name=dataset_name, settings=settings)
dataset.create()


/Users/frascuchon/.pyenv/versions/3.9.18/envs/argilla-v2/lib/python3.9/site-packages/argilla_sdk/datasets/_resource.py:84: UserWarning: Settings not provided. Using empty settings for the dataset.                     Define the settings before creating the dataset.
  self._settings = self.__configure_settings_for_dataset(settings=settings)
/Users/frascuchon/.pyenv/versions/3.9.18/envs/argilla-v2/lib/python3.9/site-packages/argilla_sdk/datasets/_resource.py:209: UserWarning: Workspace not provided. Using default workspace: argilla id: 24266b2f-49f8-4449-883f-0ca94cf34a92
  warnings.warn(f"Workspace not provided. Using default workspace: {ws.name} id: {ws.id}")


Dataset(id=UUID('1ce53fa0-a7da-4e61-9ea3-37c882006486') inserted_at=datetime.datetime(2024, 6, 7, 9, 26, 31, 56554) updated_at=datetime.datetime(2024, 6, 7, 9, 26, 31, 308021) name='news-programmatic-labeling' status='ready' guidelines=None allow_extra_metadata=False workspace_id=UUID('24266b2f-49f8-4449-883f-0ca94cf34a92') last_activity_at=datetime.datetime(2024, 6, 7, 9, 26, 31, 308021) url=None)

In [13]:
# For assign responses to users, we need to load the existing users 
users_by_name = {user.username: user for user in client.users}
current_user = client.me


Now, we can upload the records to the new dataset. For that, we can convert the loaded records into a hf dataset which can be uploaded to the new dataset

In [14]:
def map_to_record_for_single_label(data: dict, users_by_name: dict, current_user: rg.User) -> rg.Record:
    """ This function maps a text classification record dictionary to the new Argilla record."""
    suggestions = []
    responses = []
    if data.get("prediction"):
        # From data["prediction"]
        label, score = data["prediction"]
        agent = data.get("prediction_agent")
        suggestions.append(rg.Suggestion(question_name="label", value=label, score=score, agent=agent))
    if data.get("annotation"):
        # From data[annotation] and data[annotation_agent]
        user_id = users_by_name.get(data["annotation_agent"], current_user).id
        responses.append(rg.Response(question_name="label", value=data["annotation"], user_id=user_id))

    return rg.Record(
        id=data["id"],
        fields=data["inputs"],
        # The inputs field should be a dictionary with the same keys as the `fields` in the settings
        metadata=data["metadata"],
        # The metadata field should be a dictionary with the same keys as the `metadata` in the settings
        vectors=[rg.Vector(name=name, values=value) for name, value in data.get("vectors", {}).items()],
        # The vectors field should be a dictionary with the same keys as the `vectors` in the settings
        suggestions=suggestions,
        responses=responses,
    )

In [15]:
def map_to_record_for_multi_label(data: dict, users_by_name: dict, current_user: rg.User) -> rg.Record:
    suggestions = []
    responses = []
    if data.get("prediction"):
        # From data["prediction"]
        labels, scores = zip(*data["prediction"])
        agent = data.get("prediction_agent")
        suggestions.append(rg.Suggestion(question_name="labels", value=labels, score=scores, agent=agent))
    if data.get("annotation"):
        # From data[annotation] and data[annotation_agent]
        user_id = users_by_name.get(data["annotation_agent"], current_user).id
        responses.append(rg.Response(question_name="label", value=data["annotation"], user_id=user_id))

    return rg.Record(
        id=data["id"],
        fields=data["inputs"],
        # The inputs field should be a dictionary with the same keys as the `fields` in the settings
        metadata=data["metadata"],
        # The metadata field should be a dictionary with the same keys as the `metadata` in the settings
        vectors=[rg.Vector(name=name, values=value) for name, value in data.get("vectors", {}).items()],
        # The vectors field should be a dictionary with the same keys as the `vectors` in the settings
        suggestions=suggestions,
        responses=responses,
    )


In [16]:
records = []
for data in hf_records:
    records.append(map_to_record_for_single_label(data, users_by_name, current_user))

# 4. Upload the records to the new dataset
dataset.records.log(records)

[06/07/24 11:26:39] WARNING  WARNING:root:DatasetRecords: The provided batch size 256 was normalized.    ]8;id=502480;file:///Users/frascuchon/.pyenv/versions/3.9.18/envs/argilla-v2/lib/python3.9/site-packages/argilla_sdk/_helpers/_log.py\_log.py]8;;\:]8;id=917378;file:///Users/frascuchon/.pyenv/versions/3.9.18/envs/argilla-v2/lib/python3.9/site-packages/argilla_sdk/_helpers/_log.py#34\34]8;;\
                             Using value 100.                                                                      

[Record(id=003204cf-07ac-4b93-b10f-03a3b9993042,fields={'text': 'Russians admit airliner bombing blunder Russian security forces were facing further criticism last night after it was revealed that the two female Chechen suicide bombers who destroyed two planes in August with the loss '},metadata={'split': 'test'},suggestions={},responses={'label': [{'value': 'World'}]}),
 Record(id=0083a2e9-0650-4d44-8cbe-0614f2b8e64c,fields={'text': 'War crimes hearings to begin for 4 at Guantanamo US NAVAL BASE GUANTANAMO BAY, Cuba -- Four suspected Al Qaeda fighters will be formally charged with war crimes this week as the US military opens the first legal hearings for foreign prisoners captured during the war in Afghanistan and held at a remote US Navy base in Cuba.'},metadata={'split': 'test'},suggestions={},responses={'label': [{'value': 'World'}]}),
 Record(id=00a5bb0c-3810-466c-b597-b5eb239baf3b,fields={'text': 'Notable quotes Tuesday at the Athens Olympics  quot;It hurt like hell. I could see 

## Notes about the migration workflow

1. We only need a subset of functions from v1 SDK
    - `rg.init`
    - `rg.load_dataset_settings`
    - `rg.load`
   But, event like this, it would be nice to provide the whole SDK, so users can migrate iteratively their code to the v2 version.
2. Users should have a clear notion of the dataset structure (extra inputs, vectors, or metadata).
3. Created responses have a Draft status by default. There is no way to preset the status for a response, which can be blocking, even more if bulk operations in UI cannot be applied to the Draft queue.
4. When creating a rg.Record we need to prepare all about suggestions and responses before create the record itself. It could help if we can add responses or suggestions to an existing record (record.suggestions.add(...), record.responses.add(...))